
# 🐼 Pandas — Manipulation de données
_Master IA-GI — Notebook 6_

Ce notebook suit votre support **Pandas** et va en profondeur : structures (`Series`, `DataFrame`), E/S, sélection, nettoyage, agrégations, jointures, reshape (pivot/melt), temps & fenêtres, performance, stylisation, et export.  
Exemples concrets, **exercices avec hints/solutions**, et **mini‑projet** de bout en bout.

**Objectifs d’apprentissage**
- Charger, inspecter, nettoyer et transformer des données de manière idiomatique
- Sélectionner efficacement (`loc`, `iloc`, masques), typer et convertir (`astype`, `to_datetime`)
- Résumer (`groupby`, `agg`, `pivot_table`), reshaper (`melt`, `stack/unstack`), fusionner (`merge/join`)
- Travailler avec **dates**, **fenêtres mobiles**, **catégorielles** et **chaînes**
- Optimiser (vectorisation, `categorical`, `chunksize`) et exporter proprement

**Pré-requis** : NumPy, Matplotlib, notions de fichiers CSV/JSON/Excel  
**Durée estimée** : 3–4h



---
## 0) ⚙️ Préparation


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print("Pandas:", pd.__version__)
rng = np.random.default_rng(42)

# Options d'affichage utiles
pd.set_option("display.max_rows", 12)
pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 120)



---
## 1) Structures de base : `Series` & `DataFrame`


In [ ]:

s = pd.Series([10, 20, 30], index=["a","b","c"], name="score")
df = pd.DataFrame({"name":["Ali","Sara","Yasmin"], "age":[21,23,22], "grade":[14.5,16.0,12.0]})
s, df



**À retenir**
- `Series` = 1D (valeurs + index) ; `DataFrame` = 2D (colonnes nommées + index)
- `df.info()`, `df.dtypes`, `df.shape`, `df.head()`, `df.describe()`


In [ ]:

df.info()
df.describe(include="all")



---
## 2) Entrée/Sortie & typage
- `read_csv`, `read_json`, `read_excel` ; `to_csv`, `to_json`, `to_excel`
- Conversion de types : `astype`, `to_numeric`, `to_datetime`, `Categorical`


In [ ]:

# CSV jouet
df.to_csv("students.csv", index=False)
df_csv = pd.read_csv("students.csv")
# JSON
df.to_json("students.json", orient="records", force_ascii=False)
df_json = pd.read_json("students.json")

# Typage
df_csv["age"] = df_csv["age"].astype("int64")
df_csv["grade"] = pd.to_numeric(df_csv["grade"], errors="coerce")
df_csv.dtypes, df_json.head(3)



**Exercice 2.1 — Datetime & catégorielles**  
Crée un DataFrame `sales` avec colonnes `date` (chaîne), `product` (str), `qty` (int). Convertis `date` en `datetime64[ns]` et `product` en `category` ordonné par fréquence décroissante.

<details>
<summary>💡 Hint</summary>
`pd.to_datetime`, `value_counts().index`, `astype(pd.CategoricalDtype(categories=..., ordered=True))`
</details>
<details>
<summary>✅ Solution</summary>

```python
sales = pd.DataFrame({
    "date": ["2025-01-02","2025-01-05","2025-02-10","2025-01-02","2025-03-01"],
    "product": ["A","B","A","C","B"],
    "qty": [3,2,4,1,5]
})
sales["date"] = pd.to_datetime(sales["date"])
order = sales["product"].value_counts().index
cat = pd.api.types.CategoricalDtype(categories=order, ordered=True)
sales["product"] = sales["product"].astype(cat)
sales.dtypes, sales.head()
```
</details>



---
## 3) Sélection & indexation : `loc`, `iloc`, masques


In [ ]:

students = pd.DataFrame({
    "name": ["Ali","Sara","Yasmin","Youssef","Aya"],
    "age": [21, 23, 22, 24, 22],
    "grade": [14.5, 16.0, 12.0, 13.0, 16.0],
    "city": ["Fès","Meknès","Khouribga","Rabat","Casablanca"]
}).set_index("name")

# loc/iloc
subset_label = students.loc[["Sara","Aya"], ["age","grade"]]
subset_pos = students.iloc[1:4, 0:2]
mask = (students["grade"] >= 14) & (students["age"] <= 23)
filtered = students[mask]
subset_label, subset_pos, filtered



**Bonnes pratiques**
- Préférer `loc`/`iloc` plutôt que `[]` ambigu
- Chaînage d’indexation à éviter : utiliser `.loc[mask, "col"] = ...`



---
## 4) Nettoyage & préparation
- Valeurs manquantes : `isna`, `fillna`, `dropna`
- Chaînes : `.str.strip().lower().replace(...)`, regex avec `.str.replace(..., regex=True)`
- Duplicats : `duplicated`, `drop_duplicates`
- Bornes/outliers : `clip`, `quantile`, `mask`


In [ ]:

raw = pd.DataFrame({
    "id":[1,2,3,3,4],
    "city":["  Fès ","MEKNÈS","khouribga","Rabat",None],
    "score":[10, None, 200, 15, 14]
})
clean = raw.copy()
clean["city"] = (clean["city"]
                 .fillna("unknown")
                 .str.strip().str.lower()
                 .str.normalize("NFKD").str.encode("ascii","ignore").str.decode("utf-8"))
# Outliers : cap entre 0 et 100
clean["score"] = pd.to_numeric(clean["score"], errors="coerce").clip(lower=0, upper=100)
clean = clean.drop_duplicates(subset=["id"])
raw, clean



**Exercice 4.1 — Nettoyage texte**  
Standardise une colonne `country` en minuscule, sans accents, et remplace toute chaîne vide par `NaN` puis impute par `"n/a"`.

<details>
<summary>✅ Solution</summary>

```python
tmp = pd.DataFrame({"country":[" Maroc ","france","","CÔTE D'IVOIRE",None]})
tmp["country"] = (tmp["country"].replace("", np.nan)
                  .str.strip().str.lower()
                  .str.normalize("NFKD").str.encode("ascii","ignore").str.decode("utf-8")
                 ).fillna("n/a")
tmp
```
</details>



---
## 5) Transformations : vectorisation, `apply`, `map`, `assign`, `pipe`


In [ ]:

dfx = students.reset_index().copy()
dfx["passed"] = dfx["grade"] >= 10
dfx = dfx.assign(grade2=lambda d: d["grade"]**2)

# map sur séries (ex: barèmes)
bar = {True:"OK", False:"KO"}
dfx["status"] = dfx["passed"].map(bar)

# apply sur lignes (à éviter si possible, mais utile parfois)
def label_row(row):
    return f"{row['name']} ({row['city']})"
dfx["label"] = dfx.apply(label_row, axis=1)

# pipe pour enchaîner proprement
def norm_minmax(d, col):
    m, M = d[col].min(), d[col].max()
    d[col+"_norm"] = (d[col]-m)/(M-m)
    return d
dfx = (dfx.pipe(norm_minmax, col="grade")
          .pipe(norm_minmax, col="age"))

dfx.head()



---
## 6) Regrouper, agréger, fenêtrer
- `groupby(...).agg({...})`, `transform`
- Fenêtres mobiles : `rolling`, `expanding`
- Séries temporelles : `resample` (nécessite un index datetime)


In [ ]:

sales = pd.DataFrame({
    "date": pd.date_range("2025-01-01", periods=20, freq="D"),
    "product": rng.choice(list("ABC"), size=20),
    "qty": rng.integers(1, 6, size=20),
    "price": rng.normal(100, 10, size=20).round(2)
})
sales["amount"] = sales["qty"] * sales["price"]

g = (sales.groupby("product")
          .agg(qty_sum=("qty","sum"),
               amount_mean=("amount","mean"),
               n=("qty","size"))
          .sort_values("qty_sum", ascending=False))

# Rolling 7 j sur la somme des montants par jour
daily = sales.set_index("date").resample("D")["amount"].sum()
roll = daily.rolling(7, min_periods=1).mean()
g, daily.head(), roll.head(10)



**Exercice 6.1 — Transform**  
Ajoute à `sales` une colonne `zscore_amount` calculée **par produit** (centrée-réduite au sein de chaque groupe).

<details>
<summary>✅ Solution</summary>

```python
def zscore(x):
    return (x - x.mean()) / (x.std(ddof=0) + 1e-12)
sales["zscore_amount"] = sales.groupby("product")["amount"].transform(zscore)
sales.head()
```
</details>



---
## 7) Fusionner & concaténer
- `pd.merge` (équijoin, left/right/outer, clés multiples)
- `DataFrame.join` (index)
- `pd.concat` (empiler lignes/colonnes)
- As-of merges temporels : `pd.merge_asof`


In [ ]:

left = pd.DataFrame({"id":[1,2,3,4], "city":["fes","rabat","meknes","casa"]})
right = pd.DataFrame({"id":[1,1,2,5], "income":[2000,2100,2500,3000]})
m1 = pd.merge(left, right, on="id", how="left")

# concat lignes
extra = pd.DataFrame({"id":[6], "city":["tanger"], "income":[2200]})
m2 = pd.concat([m1, extra], ignore_index=True)

# merge_asof (appariement par date la plus proche <=)
ticks = pd.DataFrame({"t": pd.to_datetime(["2025-01-01","2025-01-03","2025-01-05"]),
                      "px":[10, 11, 13]})
trades = pd.DataFrame({"t": pd.to_datetime(["2025-01-02","2025-01-04"]),
                       "qty":[100, 80]})
asof = pd.merge_asof(trades.sort_values("t"),
                     ticks.sort_values("t"),
                     on="t",
                     direction="backward")
m1.head(), m2.tail(3), asof



**Exercice 7.1 — Clés multiples**  
Fais un `merge` sur `["product","date"]` entre deux DataFrames (`sales` et un `promo` listant des remises par produit/date). Remplis `NaN` de remise par 0 et calcule `amount_net = amount * (1 - discount)`.

<details>
<summary>✅ Solution</summary>

```python
promo = pd.DataFrame({
    "date": pd.date_range("2025-01-03", periods=5, freq="3D"),
    "product": ["A","B","A","C","B"],
    "discount": [0.10, 0.15, 0.05, 0.20, 0.0]
})
tmp = pd.merge(sales, promo, on=["product","date"], how="left")
tmp["discount"] = tmp["discount"].fillna(0.0)
tmp["amount_net"] = tmp["amount"] * (1 - tmp["discount"])
tmp.head()
```
</details>



---
## 8) Reshape : `pivot`, `melt`, `stack/unstack`, `pivot_table`


In [ ]:

wide = sales.pivot_table(index="date", columns="product", values="amount", aggfunc="sum")
long = wide.reset_index().melt(id_vars="date", var_name="product", value_name="amount")
stacked = wide.stack()             # MultiIndex (date, product)
unstacked = stacked.unstack()      # revient à wide
wide.head(), long.head(), stacked.head(3)



---
## 9) Index & `MultiIndex`
- Définir/réinitialiser l’index : `set_index`, `reset_index`
- MultiIndex pour hiérarchies : groupby multi-niveaux, accès `xs`


In [ ]:

mi = (sales.assign(month=lambda d: d["date"].dt.to_period("M").dt.to_timestamp())
           .set_index(["month","product"])
           .sort_index())
agg = mi.groupby(level=["month","product"])["amount"].sum()
xs_A = agg.xs("A", level="product")
agg.head(6), xs_A.head(3)



---
## 10) Séries temporelles : `resample`, `shift/diff`, fenêtres


In [ ]:

ts = (sales.set_index("date")
            .groupby("product")["amount"]
            .resample("W").sum()
            .unstack(0))  # colonnes = produits
ma = ts.rolling(2, min_periods=1).mean()
chg = ts.diff()
ts.head(), ma.head(), chg.head()



---
## 11) Performance & grands jeux de données
- Catégorielles (`astype('category')`) pour colonnes discrètes
- Vectoriser au maximum ; éviter les `apply` ligne
- Lecture par morceaux : `read_csv(..., chunksize=10_000)` + `concat`


In [ ]:

big = pd.DataFrame({
    "k": rng.choice(list("ABCDEFGHIJ"), size=50_000),
    "v": rng.normal(0, 1, size=50_000)
})
big_cat = big.assign(k=lambda d: d["k"].astype("category"))
res = big_cat.groupby("k")["v"].mean().sort_values(ascending=False).head(5)
big.memory_usage(deep=True).sum(), big_cat.memory_usage(deep=True).sum(), res



---
## 12) Visualisation rapide (via Matplotlib)
> Pandas délègue à Matplotlib. Un **graphique par figure** pour respecter les consignes.


In [ ]:

daily.plot()
plt.title("Montant quotidien (toutes catégories)")
plt.xlabel("Date"); plt.ylabel("Montant")
plt.tight_layout()


In [ ]:

ma.plot()
plt.title("Moyenne mobile hebdo (fenêtre=2)")
plt.xlabel("Semaine"); plt.ylabel("Montant moyen")
plt.tight_layout()



---
## 13) Mise en forme tabulaire & export
- `DataFrame.style` pour surligner, barre de progression, formats
- Export : `to_csv`, `to_excel`, `to_json`, `to_parquet` (si installé)


In [ ]:

top = (sales.groupby("product")["amount"]
             .sum()
             .sort_values(ascending=False)
             .rename("revenue"))

styled = (top.to_frame()
            .style.format("{:,.2f}")
            .bar(color=None, vmin=0)  # sans préciser de couleur
         )
styled


In [ ]:

# Exports
sales.to_csv("sales_clean.csv", index=False)
wide.to_excel("sales_pivot.xlsx")
"Fichiers enregistrés: sales_clean.csv, sales_pivot.xlsx"



---
## 14) 📝 Quiz express
1) Différence entre `loc` et `iloc` ?  
2) `groupby(...).transform` vs `agg` ?  
3) Quand utiliser `merge_asof` ?  
4) Deux moyens de reshaper de long à large ?  
5) Avantages des variables catégorielles ?

<details>
<summary>✅ Corrigé</summary>

1) `loc` par **labels** ; `iloc` par **positions**.  
2) `transform` renvoie un **résultat aligné** sur la taille d’origine ; `agg` résume **par groupe**.  
3) Appariement temporel **par plus proche ≤** (timestamps).  
4) `pivot/pivot_table` et `unstack` (ou `melt` dans l’autre sens).  
5) **Mémoire réduite**, perfs groupby/tri, ordres logiques garantis.
</details>



---
## 15) 🎯 Mini‑projet — *Pipeline de données “ventes” complet*
**Contexte** : Tu disposes de fichiers `sales.csv` (transactions) et `products.csv` (catalogue).  
**Objectif** : Construire un pipeline **reproductible** : nettoyage ➜ enrichissement ➜ KPIs ➜ reshape ➜ séries temporelles ➜ export.

**Spécifications**
1) **Chargement** : simuler `sales.csv` (date, product_id, qty, price) et `products.csv` (product_id, name, category).  
2) **Nettoyage** : `to_datetime`, outliers sur `price` (clip quantiles 1%–99%), suppression duplicats.  
3) **Enrichissement** : `amount = qty*price`, jointure sur `product_id`.  
4) **KPIs** : par `category` ➜ revenu total, panier moyen (`amount/transaction`), top‑N produits.  
5) **Reshape** : pivot mensuel `category × month` des revenus, plus `rolling(2)` sur la série globale.  
6) **Export** : `to_csv` des tables clés + 2 figures Matplotlib (un graphe/figure).

> Tu peux réutiliser `pipe`, `assign`, `groupby`, `merge`, `pivot_table`, `resample` et `rolling`.


In [ ]:

# 1) Chargement (simulation)
rng = np.random.default_rng(123)
n = 500
dates = pd.to_datetime("2025-01-01") + pd.to_timedelta(rng.integers(0, 120, n), unit="D")
product_ids = rng.integers(100, 110, n)
qty = rng.integers(1, 6, n)
price = np.round(rng.normal(100, 20, n), 2)

sales = pd.DataFrame({"date": dates, "product_id": product_ids, "qty": qty, "price": price})
products = pd.DataFrame({
    "product_id": np.arange(100,110),
    "name": [f"P{i}" for i in range(100,110)],
    "category": rng.choice(["A","B","C"], size=10)
})

# 2) Nettoyage
sales["date"] = pd.to_datetime(sales["date"], errors="coerce")
q1, q99 = sales["price"].quantile([0.01, 0.99])
sales["price"] = sales["price"].clip(q1, q99)
sales = sales.drop_duplicates()

# 3) Enrichissement
sales["amount"] = sales["qty"] * sales["price"]
full = pd.merge(sales, products, on="product_id", how="left")

# 4) KPIs
revenue_by_cat = full.groupby("category")["amount"].sum().sort_values(ascending=False)
basket = (full.assign(basket=lambda d: d["amount"])  # une ligne = une transaction
               .groupby("category")["basket"].mean())

topN = (full.groupby(["category","name"])["amount"].sum()
            .sort_values(ascending=False)
            .groupby(level=0).head(3)
            .reset_index())

# 5) Reshape mensuel + rolling
full["month"] = full["date"].dt.to_period("M").dt.to_timestamp()
pivot_rev = full.pivot_table(index="month", columns="category", values="amount", aggfunc="sum")
rev_daily = full.set_index("date")["amount"].resample("D").sum()
rev_roll = rev_daily.rolling(14, min_periods=1).mean()

# 6) Export tables
revenue_by_cat.to_csv("kpi_revenue_by_category.csv")
basket.to_csv("kpi_basket_by_category.csv")
pivot_rev.to_csv("pivot_month_category_revenue.csv")

# Figures (Matplotlib, une par figure)
pivot_rev.plot()
plt.title("Revenus mensuels par catégorie")
plt.xlabel("Mois"); plt.ylabel("Revenu")
plt.tight_layout()
plt.savefig("fig_revenue_month_category.png", dpi=300, bbox_inches="tight")

rev_roll.plot()
plt.title("Revenu quotidien (moyenne mobile 14j)")
plt.xlabel("Date"); plt.ylabel("Revenu (MM14)")
plt.tight_layout()
plt.savefig("fig_revenue_rolling.png", dpi=300, bbox_inches="tight")

# Résumé
revenue_by_cat.head(), basket.head(), topN.head(), pivot_rev.head()



---
## 📚 Ressources
- Pandas User Guide : https://pandas.pydata.org/docs/user_guide/index.html  
- Cookbook : https://pandas.pydata.org/docs/user_guide/cookbook.html  
- Time series : https://pandas.pydata.org/docs/user_guide/timeseries.html  
- Reshaping : https://pandas.pydata.org/docs/user_guide/reshaping.html  
